In [46]:
from dotenv import load_dotenv
load_dotenv()


True

In [47]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, ChatVectorDBChain


In [48]:
from langchain.document_loaders import TextLoader, WebBaseLoader

jardin = WebBaseLoader("https://buenosaires.gob.ar/jardin-botanico")
carnaval = TextLoader("/Users/alexander/carnaval.txt")




In [49]:
jardin_doc = jardin.load()

In [50]:
carnaval_doc = carnaval.load()

In [51]:
documents = jardin_doc + carnaval_doc

In [52]:
documents

[Document(page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\nJardín Botánico | Buenos Aires Ciudad - Gobierno de la Ciudad Autónoma de Buenos Aires\n\n\n\n\n\n\n\n\n\n\n\n\n\n      Pasar al contenido principal\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\nInicio\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBuenos Aires\n\n\n\n\n\n\n\nJardín Botánico \n\n\n\n\nLas más diversas especies vegetales crecen y se reproducen en uno de los jardines más importantes de Buenos Aires\n\n\n\n\n\n\n\nInstitucional\n\n\nQuiénes somos\n\n\nInformación General\n\n\n\n\nHistoria\n\n\nLos jardines botánicos y de aclimatación\n\n\nBiografía Julio Carlos Thays\n\n\nMaestros de la Botánica\n\n\nUn edificio con historia\n\n\n\n\nMapa\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n      Mujeres en la Botánica\n    \n\n\n\n\n\n\n      El cambio climático ya sucede\n    \n\n\n\n\n\n\n      Los suelos secos toman menos agua\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nattractions\nActividades\n\n      Informate s

In [53]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [54]:
qa = ChatVectorDBChain.from_llm(llm=OpenAI(temperature=0),vectorstore=docsearch,return_source_documents=False)



In [55]:
vectordbkwargs = {"search_distance": 0.9}

In [58]:
chat_history = []
query = "a que hora son los corsos?"
result = qa({"question": query, "chat_history": chat_history, "vectordbkwargs": vectordbkwargs})



In [59]:
result

{'question': 'a que hora son los corsos?',
 'chat_history': [],
 'vectordbkwargs': {'search_distance': 0.9},
 'answer': ' Los corsos se realizan en los fines de semana de febrero, los sábados, el domingo 19 y lunes 20 en el horario de 19 a 02 h., y domingos de 19 a 00 h.'}

In [60]:
chat_history = [(query, result["answer"])]
query = "y donde son?"
result = qa({"question": query, "chat_history": chat_history, "vectordbkwargs": vectordbkwargs})


In [61]:
result['answer']

' Los corsos se realizan en 33 locaciones elegidas en diálogo con la comunidad carnavalera. Estos incluyen los corsos emblemáticos Boedo 1 y 2, Villa Puerredon, Flores, Mataderos, Saavedra, Villa Urquiza, Puente Saavedra, La Boca, Villa Crespo y Villa Devoto, así como los corsos normales Paternal 1 y Agronomía.'

In [63]:
chat_history = [(query, result["answer"])]
query = "en boedo hay alguno?"
result = qa({"question": query, "chat_history": chat_history, "vectordbkwargs": vectordbkwargs})


In [64]:
chat_history

[('y donde son?',
  ' Los corsos se realizan en 33 locaciones elegidas en diálogo con la comunidad carnavalera. Estos incluyen los corsos emblemáticos Boedo 1 y 2, Villa Puerredon, Flores, Mataderos, Saavedra, Villa Urquiza, Puente Saavedra, La Boca, Villa Crespo y Villa Devoto, así como los corsos normales Paternal 1 y Agronomía.')]

In [65]:
result['answer']

' Sí, hay dos corsos en Boedo. Boedo 1 tendrá lugar los días 4, 11, 18, 19 y 20 de febrero, y Boedo 2 tendrá lugar los días 4, 5, 11 y 12 de febrero.'